# Coarse-Grained Polymer

This tutorial will give a basic example of creating a custom coarse-grained model in OpenMM. It demonstrates how to create a custom forcefield xml file and how to create a molecular topology from scratch using the OpenMM python API.
It assumes you are familiar with the concept of coarse-graining and want to learn how to implement your CG model in OpenMM.

## Example System
The example system in this tutorial is a Lennard-Jones bead-spring polymer model. We will demonstrate how to create a polymer melt of multiple chains. Note that we will set the bond lengths, particle mass, and Lennard-Jones interaction parameters to physical values that are typical of a residue level coarse-grained protein model (e.g. [xxx]).

## Creating the force-field file
We will begin by creating a custom force-field xml file for our system. You should look here first for the full information: http://docs.openmm.org/latest/userguide/application/05_creating_ffs.html

An OpenMM forcefield is defined by an XML file. We will create a new custom one for this tutorial. You will need to create a new file called "cg_ff.xml" using a text editor of your choice. Then paste the following lines into it:

```xml
<!-- cg_ff.xml 
Coarse-grained forcefield for a bead-spring polymer. -->
<ForceField>
	
    <AtomTypes>
	    <Type name="CG-bead" class="CG-bead" element="CG" mass="120.0"/>
	</AtomTypes>

	<Residues>
        <!-- Each bead is a single residue.
        Need a template for the different Residue types. 
        First type is the end. This only has one bond. -->
        <Residue name="END"> -->
            <Atom name="CG-bead" type="CG-bead"/>
            <ExternalBond atomName="CG-bead"/>
        </Residue>

        <!-- Second type is in the middle of the chain. This has two bonds. -->
        <Residue name="MID">
            <Atom name="CG-bead" type="CG-bead"/>
            <ExternalBond atomName="CG-bead"/>
            <ExternalBond atomName="CG-bead"/>
        </Residue>
    </Residues>

    <!-- Define a harmonic bond between the CA-beads -->
    <HarmonicBondForce>
        <Bond class1="CG-bead" class2="CG-bead" length="0.38" k="1000.0"/>
    </HarmonicBondForce>

    <!-- Use a custom non-bonded force for maximum flexibility.
    The bondCutoff=1 tells it to only exclude interactions between directly bonded atoms. -->
    <CustomNonbondedForce energy="4*epsilon*((sigma/r)^12-(sigma/r)^6); sigma=0.5*(sigma1+sigma2); epsilon=sqrt(epsilon1*epsilon2)" bondCutoff="1">
        <PerParticleParameter name="sigma"/>
        <PerParticleParameter name="epsilon"/>
		<Atom type="CG-bead" charge="0.0" sigma="0.5" epsilon="1.0"/>
	</CustomNonbondedForce>
</ForceField>
```
(If you have cloned the cookbook repo then this file will already exist)
The comments in the above code explain what the difference sections are for.


## Creating CG Topology
Now that we have defined a force-field we can use we can create the topology of the system. To demonstrate the full flexibility open OpenMM we will use the Python API to do this. Note that you could create a PDB file and read that in instead.

First we do the usual imports for OpenMM.

In [ ]:
import openmm
import openmm.app as app
import openmm.unit as unit
import numpy as np
from sys import stdout

We then create a new element for our CG bead. This needs to have the same symbol as the element we defined in the `<AtomTypes>` section of 'cg_ff.xml'. We set the atomic index to a large number, however the actual value is unimportant in this simulation and is just used for bookkeeping purposes. We set the mass to approximate the molar mass of an amino-acid. The mass we define is important as it needs to be consistent with the chosen integration timestep.

In [ ]:

cgElement = app.Element(number=1000,name='CG-element',symbol='CG', mass=120)

We then create an empty `Topology` object.

In [ ]:
# Make an empty topology
topology = app.Topology()

Now we can create our simulation topology using python scripting. We will define `M = 100` polymer chains which each have `N = 10` beads:

In [ ]:
# Number of polymer chains
M = 100

# Number of atoms in each chain
N = 10

# Add each chain to the topology
for m in range(M):

    # Make the chain
    chain = topology.addChain()

    # Create the first residue in the chain
    residue = topology.addResidue(name="END", chain=chain)

    # In this example each residue is one bead so we add a single atom
    atom1 = topology.addAtom(name="CG-bead", element=cgElement, residue=residue)

    # Now add the rest of residues in the chain
    for i in range(1, N-1):
        residue = topology.addResidue(name="MID",chain=chain)
        atom2 = topology.addAtom(name="CG-bead", element=cgElement, residue=residue)

        # add the bonds in as we go
        topology.addBond(atom1, atom2)
        atom1 = atom2
    
    # Add the final residue in the chain
    residue = topology.addResidue(name="END",chain=chain)
    atom2 = topology.addAtom(name="CG-bead", element=cgElement, residue=residue)
    topology.addBond(atom1, atom2)


topology.setPeriodicBoxVectors(np.eye(3)*11.0*unit.nanometers)

# check the topology
print(topology)

## Create the system

We can now load in our previously created custom `ForceField` and use the `createSystem` method with the `topology`.

In [ ]:
# load in the ForceField we created
ff = app.ForceField('./cg_ff.xml')
system = ff.createSystem(topology, nonbondedMethod=openmm.app.CutoffPeriodic, nonbondedCutoff=1.5*unit.nanometers)

We have set a nonbonded cutoff to be 3 times the value of sigma in LJ potential.

## Setup the simulation

We will use a `LangevinMiddleIntegrator` with a friction term of 0.01ps^-1 and a timestep of 0.01ps as used in similar coarse-grained polymer models [xxx]. For your own models these parameters will be important and you will need to choose them carefully!


In [ ]:
integrator = openmm.LangevinMiddleIntegrator(300*unit.kelvin, 0.01/unit.picosecond, 0.010*unit.picoseconds)
simulation = app.Simulation(topology, system, integrator)

## Assign the initial positions

As we have created the topology from scratch we will also need to define the initial positions from scratch. We will arrange our 100 polymer chains in a 10x10 grid in the X,Y plane and each polymer will be in a linear configuration in the Z direction.

In [ ]:
positions = []
for m in range(M):
    x0 = np.array(((m%10)*1.0, (m//10)*1.0, 0))*unit.nanometers
    positions.append(x0)
    for i in range(1,N):
        xi = positions[-1] + np.array((0, 0, 0.38))*unit.nanometers
        positions.append(xi)

assert(len(positions) == topology.getNumAtoms())
simulation.context.setPositions(positions)

The initial configuration looks like this:
![initial configuration](initial_config.png)


## Equilibrate the simulation

We will now setup the simulation reporters, run NVT equilibration, and run NPT equilibration. 


In [ ]:
# setup simulation reporters
# Write the trajectory to a file called 'traj.dcd'
simulation.reporters.append(app.DCDReporter('traj.dcd', 1000, enforcePeriodicBox=False))

# Report information to the screen as the simulation runs
simulation.reporters.append(app.StateDataReporter(stdout, 1000, step=True,
        potentialEnergy=True, temperature=True, volume=True, speed=True))


# output the initial configuration
with open('initial_config.pdb','w') as f:
    app.PDBFile.writeFile(topology, simulation.context.getState(getPositions=True).getPositions(), f)

# Now we can run NVT equilibration
simulation.step(10000)

# Add a barostat
barostatIndex=system.addForce(openmm.MonteCarloBarostat(1.0*unit.bar, 300*unit.kelvin))
simulation.context.reinitialize(preserveState=True)

# Run NPT equilibration
simulation.step(200000)


# output the equilibrated configuration
with open('equilibrated_config.pdb','w') as f:
    state = simulation.context.getState(getPositions=True, enforcePeriodicBox=True)
    topology.setPeriodicBoxVectors(state.getPeriodicBoxVectors())
    app.PDBFile.writeFile(topology, state.getPositions(), f)

The equilibrated system will be a polymer melt that looks similar to this:
![equilibrated polymer melt](equilibrated_config.png)

## References